In [1]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!mkdir -p ~/data/flowers
!tar -xzf 102flowers.tgz -C ~/data/flowers/
!rm 102flowers.tgz
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat
!mv setid.mat ~/data/flowers/setid.mat
!mv imagelabels.mat ~/data/flowers/imagelabels.mat

--2020-04-03 10:05:46--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344862509 (329M) [application/x-gzip]
Saving to: ‘102flowers.tgz’

102flowers.tgz      100%[===================>] 328.89M  32.5MB/s    in 11s     

2020-04-03 10:05:57 (30.6 MB/s) - ‘102flowers.tgz’ saved [344862509/344862509]

--2020-04-03 10:06:09--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14989 (15K)
Saving to: ‘setid.mat’

setid.mat           100%[===================>]  14.64K  --.-KB/s    in 0.09s   

2020-04-03 10:06:09 (164 KB/s) - ‘setid.mat’ saved [14989/14989]



In [2]:
!ls ~/data/flowers

imagelabels.mat  jpg  setid.mat


In [3]:
cd ~/data/flowers/

/root/data/flowers


In [4]:
import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy.io
from keras.utils import to_categorical
import cv2
from sklearn.model_selection import train_test_split

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [0]:
img_labels = scipy.io.loadmat("imagelabels.mat")
img_labels = img_labels["labels"]
img_labels = img_labels[0]
for i in range(len(img_labels)):
    img_labels[i] = img_labels[i] - 1

In [0]:
splits = scipy.io.loadmat("setid.mat")
test_data = splits["valid"][0]

In [7]:
print(len(test_data))

1020


In [0]:
test_x = []
dir = "jpg/"
for imgs in test_data:
    image = str(imgs).zfill(5)
    image = cv2.imread(os.path.join(dir,"image_"+image+".jpg"))
    resized = cv2.resize(image, (150,150))
    normalized_img = cv2.normalize(resized, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    test_x.append(normalized_img)

In [0]:
test_x = np.array(test_x)
test_y = []
for i in range(len(test_x)):
    test_y.append(img_labels[test_data[i]-1])

In [10]:
test_x.shape

(1020, 150, 150, 3)

In [13]:
graph1 = tf.Graph()
with graph1.as_default():
  with tf.Session(graph=graph1) as sess:
    tf.saved_model.loader.load(sess, ["serve"], '/root/data/flowers')
    x = graph1.get_tensor_by_name('x:0')
    layer_fc2 = graph1.get_tensor_by_name('layer_fc2:0')
    preds = sess.run(layer_fc2, feed_dict={x: test_x})

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=tf.cast(preds, tf.float32),
                                                              labels=tf.cast(tf.keras.utils.to_categorical(test_y), tf.float32)))
    accuracy = tf.reduce_mean(tf.cast(tf.equal(np.argmax(preds, axis=1), test_y), tf.float32))

    print('Loss :',loss.eval())
    print('Accuracy :', accuracy.eval())



INFO:tensorflow:Restoring parameters from /root/data/flowers/variables/variables
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Loss : 1.4947926
Accuracy : 0.66764706
